# MultiLayer Perceptron for inpainting one pixel an image using tensorflow 1

In [14]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
from data_generators import gen_frm_pic
from util_functions import util_functions

INDEX_TRGT = 4
INDEX_END = 8
ROW_SHAPE = (1,9)

## Generate images data

In [15]:
data = gen_frm_pic.gen_frm_pic.generate()

[array([[[10,  6,  0],
        [24, 20,  9],
        [28, 24, 13]],

       [[24, 20,  9],
        [42, 38, 27],
        [48, 44, 35]],

       [[28, 24, 13],
        [48, 44, 33],
        [59, 52, 42]]], dtype=uint8), array([[[17, 13,  4],
        [ 9,  4,  0],
        [15, 10,  4]],

       [[32, 28, 19],
        [17, 10,  2],
        [17, 10,  0]],

       [[40, 33, 23],
        [17,  9,  0],
        [11,  4,  0]]], dtype=uint8), array([[[26, 17, 12],
        [32, 18,  9],
        [54, 36, 16]],

       [[27, 17,  5],
        [36, 23,  6],
        [51, 31,  4]],

       [[23, 12,  0],
        [35, 21,  0],
        [54, 36,  0]]], dtype=uint8), array([[[ 97,  72,  42],
        [120,  91,  49],
        [121,  89,  38]],

       [[ 89,  65,  27],
        [114,  86,  39],
        [125,  94,  40]],

       [[ 81,  57,  11],
        [101,  71,  21],
        [116,  85,  30]]], dtype=uint8), array([[[120,  91,  35],
        [128, 102,  45],
        [120,  98,  41]],

       [[131, 100,  45]

## Split each pixel data to RGB channels

In [16]:
# red channel data
r_data = []

# green channel data
g_data = []

# blue channel data
b_data = []
j = 0 
for image in data:
    for i in range(3):
        j += 1
        if i == 0:
            r_temp = image[i].reshape(ROW_SHAPE)[0]
            util_functions.swap(r_temp, INDEX_TRGT, INDEX_END)
            r_data.append(r_temp)

        if i == 1:
            g_temp = image[i].reshape(ROW_SHAPE)[0]
            util_functions.swap(g_temp, INDEX_TRGT, INDEX_END)
            g_data.append(g_temp)
            
        if i == 2:
            b_temp = image[i].reshape(ROW_SHAPE)[0]
            util_functions.swap(b_temp, INDEX_TRGT, INDEX_END)
            b_data.append(b_temp)

data = [r_data, g_data, b_data]


## Split data to train, test and vailation data

In [29]:
# for red channel
train_r_data = r_data[0:1200]
test_r_data = r_data[1200:1600]
valid_r_data = r_data[1600:2000]

# for green channel
train_g_data = g_data[0:600]
test_g_data = r_data[800:1000]
valid_g_data = r_data[600:800]

# for blue channel
train_b_data = b_data[0:600]
test_b_data = b_data[800:1000]
valid_b_data = b_data[600:800]

In [30]:
# assuming 3 regressors will give good accuracy
# prepering the data for each regressor
x_train_r = []
x_train_g = []
x_train_b = []

y_train_r = []
y_train_g = []
y_train_b = []

for i in range(1000):
    
    x_train_r.append(train_r_data[i][0:INDEX_END].tolist())
    y_train_r.append([train_r_data[i][INDEX_END]])



In [19]:
y_train_r

[[20],
 [4],
 [18],
 [91],
 [102],
 [61],
 [89],
 [57],
 [51],
 [41],
 [42],
 [71],
 [56],
 [47],
 [132],
 [82],
 [116],
 [129],
 [58],
 [101],
 [74],
 [79],
 [48],
 [55],
 [63],
 [62],
 [60],
 [107],
 [143],
 [126],
 [81],
 [82],
 [75],
 [92],
 [32],
 [16],
 [39],
 [66],
 [108],
 [42],
 [76],
 [58],
 [45],
 [40],
 [49],
 [58],
 [49],
 [39],
 [123],
 [101],
 [114],
 [97],
 [52],
 [103],
 [68],
 [47],
 [42],
 [75],
 [71],
 [89],
 [81],
 [131],
 [144],
 [127],
 [90],
 [97],
 [117],
 [117],
 [73],
 [43],
 [93],
 [53],
 [121],
 [56],
 [82],
 [77],
 [60],
 [49],
 [63],
 [44],
 [46],
 [51],
 [108],
 [68],
 [101],
 [93],
 [78],
 [56],
 [80],
 [83],
 [96],
 [72],
 [75],
 [94],
 [118],
 [128],
 [117],
 [128],
 [92],
 [94],
 [118],
 [98],
 [103],
 [100],
 [105],
 [59],
 [100],
 [79],
 [85],
 [56],
 [62],
 [58],
 [50],
 [49],
 [39],
 [64],
 [93],
 [123],
 [101],
 [55],
 [108],
 [82],
 [95],
 [88],
 [94],
 [107],
 [110],
 [130],
 [112],
 [124],
 [106],
 [114],
 [82],
 [99],
 [98],
 [101],
 [97],
 

## Model

In [43]:
features = 8 # each channel
hidden_layer_nodes = 10
x = tf.placeholder(tf.float32, [None, features])
y_ = tf.placeholder(tf.float32, [None, 1])
W1 = tf.Variable(tf.truncated_normal([features, hidden_layer_nodes]))
b1 = tf.Variable(tf.constant(0.1, shape=[hidden_layer_nodes]))
z1 = tf.add(tf.matmul(x,W1),b1)
a1 = tf.nn.relu(z1) # using relu function
W2 = tf.Variable(tf.truncated_normal([hidden_layer_nodes,1]))
b2 = tf.Variable(0.)
z2 = tf.matmul(a1,W2) + b2

loss = tf.reduce_mean(tf.pow(z2 - y_, 2))
update = tf.train.AdamOptimizer(0.001).minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(0,10000):
        sess.run(update, feed_dict = {x:x_train_r, y_:y_train_r})
        if i % 1000 == 0:
                
                print(loss.eval(session=sess, feed_dict = {x:x_train_r, y_:y_train_r}))
        elif i == 10:
                print("W1: ", sess.run(W1), "w2: ", sess.run(W2), "b: ", sess.run(b1))


        
        # sess.run(update, feed_dict = {x:data_x, y_:data_y})


3919.676
W1:  [[ 0.2717188   0.20013906  0.03758326  0.10197315  0.60394126 -0.5341603
   0.6762875   1.0812336  -0.782869   -0.4839407 ]
 [-0.43931788 -1.7053287   1.1407291   0.05041609 -0.7441394   0.7403584
  -0.11304387 -0.32219714  1.8456101  -0.26856437]
 [ 1.4749296  -0.2679236  -0.9367444  -1.047976   -1.4374993   0.7931698
  -1.0473329  -0.29145053  1.372803    0.17154728]
 [-0.57516634 -1.036655   -0.11143942 -0.37476897  0.18450111 -0.06520297
  -1.1568139  -1.1916918   0.77050024 -0.9170976 ]
 [-0.2643586   0.28388557 -0.33939284  0.34323606 -0.04019991 -1.6954314
  -0.17410752  0.5117101   0.6796392  -0.4809466 ]
 [ 0.58314383 -1.7477002   1.2284911   0.24108075 -0.31350946  0.00195231
   0.4128251  -0.04258882 -0.5824369   0.0426923 ]
 [-1.190598   -0.07664017 -0.6677936   0.10672572 -1.1283641  -0.75213784
  -0.20915046 -0.16576043 -0.8748789  -0.17190729]
 [ 0.8847478  -0.04216938 -0.59639734  0.0940719   0.61426485 -1.061497
  -0.38347346 -1.8460866  -1.0825655  -0.89

In [20]:
# print('prediction: ', y.eval(session=sess, feed_dict = 	{x:[[13,12], [0,33], [40,3], [1,1], [50,50]]}))
# sess.run(update, feed_dict = {x:x_train_r, y_:y_train_r})
print(loss.eval(session=sess, feed_dict = {x:x_train_r, y_:y_train_r}))
print("W1: ", sess.run(W1), "w2: ", sess.run(W2), "b: ", sess.run(b1))



806.986
W1:  [[-6.75322255e-03 -1.83656263e+00 -9.19409171e-02 -2.91433811e+01
  -1.38352469e-01 -7.11446106e-02]
 [-1.24929145e-01 -1.39316595e+00 -2.18483228e-02 -2.41678753e+01
   1.00743689e-01  8.11009388e-03]
 [-1.26606032e-01 -4.88137752e-01  9.69168171e-02 -1.04048471e+01
   5.02216443e-02 -7.21452311e-02]
 [ 7.12845800e-03 -1.58668482e+00  5.91390207e-02 -2.95283852e+01
  -4.00139540e-02  7.15420768e-02]
 [-3.15114446e-02 -6.21528924e-01  3.68732251e-02 -1.06751699e+01
  -9.87598598e-02  1.07530773e-01]
 [ 4.81233262e-02 -6.50106549e-01  9.22799632e-02 -1.09308853e+01
  -1.61356241e-01 -1.30413398e-02]
 [-8.92974883e-02 -1.71713829e+00 -1.58934578e-01 -2.94896660e+01
  -1.22638613e-01 -1.51780099e-01]
 [-5.55237308e-02 -1.52738404e+00 -1.32213697e-01 -2.43119946e+01
   1.43107444e-01 -1.55548099e-02]] w2:  [[-9.3923315e-02]
 [ 4.5048621e-01]
 [ 1.4945497e-01]
 [-6.6925774e+01]
 [ 9.8852105e-02]
 [ 2.2468828e-02]] b:  [ 0.1         0.07957114  0.1        -0.24920994  0.09966704